# Lesson 1: Router Engine

Welcome to Lesson 1.

To access the `requirements.txt` file, the data/pdf file required for this lesson and the `helper` and `utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [1]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio

nest_asyncio.apply()

## Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

In [3]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["storyboard.pdf"]).load_data()

In [4]:
documents[:5]

[Document(id_='ae1fbcae-d1d9-4cc8-8a83-63760cf65c40', embedding=None, metadata={'page_label': '1', 'file_name': 'storyboard.pdf', 'file_path': 'storyboard.pdf', 'file_type': 'application/pdf', 'file_size': 259171, 'creation_date': '2024-09-04', 'last_modified_date': '2024-09-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='The\tArt\tand\tEvolution\tof\tB-Roll\nI.\tIntroduction\tto\tB-Roll\nDefinition\tand\tImportance\nB-Roll\trefers\tto\tsupplemental\tor\talternative\tfootage\tintercut\twith\tthe\tmain\tshot\tin\tvideo\tproduction.\tIt\tis\tcrucial\tfor\nenhancing\tstorytelling,\tproviding\tcontext,\tand\tmaintaining\tviewer\tengagement.\tBy\tincorporating\tB-Roll,\tfilmmakers\tand\ncontent\tcreators\tcan\tillustrate\tpoints\tmore\tvividly,\tbreak\t

## Define LLM and Embedding model

In [5]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

nodes[:5]

[TextNode(id_='4488dec7-6714-4ef5-af90-0aed3c0d533d', embedding=None, metadata={'page_label': '1', 'file_name': 'storyboard.pdf', 'file_path': 'storyboard.pdf', 'file_type': 'application/pdf', 'file_size': 259171, 'creation_date': '2024-09-04', 'last_modified_date': '2024-09-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ae1fbcae-d1d9-4cc8-8a83-63760cf65c40', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'storyboard.pdf', 'file_path': 'storyboard.pdf', 'file_type': 'application/pdf', 'file_size': 259171, 'creation_date': '2024-09-04', 'last_modified_date': '2024-09-03'}, hash='3d7a5569bc21fbfec4e85e93d3cf35382a5726ffd2cacd3d4cf5e4c128646759'), <NodeRelationship.NEXT:

In [7]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-4o")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Define Summary Index and Vector Index over the Same Data

In [8]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [9]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [11]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to Story Writing, B-Rolls, and Visualization"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the Story Writing, B-Rolls, and Visualization."
    ),
)

## Define Router Query Engine

In [12]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [13]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Choice 1 is relevant because it is useful for summarization questions, which aligns with the request for a summary of the document..
The document provides a comprehensive guide on the art and evolution of B-Roll, emphasizing its importance in enhancing storytelling, providing context, and maintaining viewer engagement. It covers the historical context of B-Roll, its evolution in modern media, and its indispensable role in digital content creation. The document also delves into planning and executing effective B-Roll, including pre-production considerations like storyboarding, shot lists, location scouting, budgeting, and equipment setup. It categorizes different types of B-Roll footage, such as atmospheric shots, action shots, establishing shots, cutaway shots, and insert shots, explaining their usage and significance.

Additionally, the document explores advanced techniques for shooting effective B-Roll, including planning, shot lists, techniques and angles, 

In [14]:
print(len(response.source_nodes))

69


In [15]:
response = query_engine.query(
    "When do I add a B-Roll for a video course so that the course will be more engaging and be more educational?"
)
print(str(response))

Selecting query engine 1: The question is asking for specific context on when to add a B-Roll to make the course more engaging and educational..
To make a video course more engaging and educational, add B-Roll to illustrate key points made in the main footage, provide visual context, and maintain viewer interest. Use B-Roll to complement and enhance the main narrative without overwhelming it. For example, include B-Roll of relevant activities, environments, or objects that relate to the course content. Additionally, strategically place B-Roll to control the pacing and rhythm of the video, ensuring a dynamic and engaging viewing experience.


## Let's put everything together

In [ ]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

In [ ]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))